## Plan for continuous training

Here's the plan. The script boots up and checks a continuous play directory which is populated with a number of training round sub-directores. Each sub-directory is labelled in sequence and contains a self_play sub-sub-directory. The script takes stock of the contents of the latest training round self_play directory.

If the number of files (tournaments) saved in the self_play directory of the latest training round is less than some minimum M, say m < M then the self-play script is kicked off to play M - m more tournaments, completing the M requred self-play tournaments and saving to the  subdirectory.

If the number of files saved in the self_play directory of the latest training round m >= M then the script checks if there is already a model saved in the training round directory.

If there is not a model saved in the training round directory, the script will kick off a training routine to create a new model (or load a saved model from a possibly existing previous round), compile a training dataset from the most recent k training rounds (augmented with the catalogue of known checkmate positions) and train the model to stopping. The script will then save the model in the training round sub-directory.

Then the script will create a new training round directory, labelled with the next integer in the training round sequence, containing a self_play directory. At this point we can essentially continue the loop from the top.

In [1]:
import os, torch
from torch.utils.data import DataLoader
from chess_selfplay import harvest_checkmates
from chess_model import TransformerModel, ChessDataset, TanhLoss, train
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

min_tournaments_each_round = 50 # A new model roughly once a day.
root_dir = os.path.join('data','output')
dir_sort_key = lambda d: int(d.split('_')[-1])

## LOOP STARTS - just kill the machine any old time when you have stuff to do and you can fire it up again whenever you're ready.
while True:

    training_round_dirs = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir,d))], key=dir_sort_key)
    current_training_round_dir = training_round_dirs[-1]
    previous_training_round_dir = training_round_dirs[-2]

    current_self_play_path = os.path.join(root_dir, current_training_round_dir, 'self_play')
    if os.path.exists(current_self_play_path) and os.path.isdir(current_self_play_path):
        number_games_played_already = len(os.listdir(current_self_play_path))
    else:
        os.mkdir(current_self_play_path)
        number_games_played_already = 0

    tournaments_left_to_play = min_tournaments_each_round - number_games_played_already
    print(f'Current: {current_training_round_dir}, already played: {number_games_played_already}, left to play: {tournaments_left_to_play}')

    if tournaments_left_to_play > 0:
        
        # self-play script to play tournaments_left_to_play more tournaments, saving to current_self_play_dir.
        model_source_path = os.path.join(root_dir, previous_training_round_dir, 'model.pt')
        assert os.path.exists(model_source_path) and os.path.isfile(model_source_path), 'ERROR: MODEL NOT FOUND.'

        # Same base model with different look-ahead strength configuration 
        model_kwargs = {'nlayers':6, 'nheads':3, 'embed_dim':18, 'dk':5, 'device':device, 'load_path':model_source_path}
        agent0_spec = {'type': 'transformer', 'kwargs': model_kwargs, 'num_simgames': 200, 'max_simmoves': 5, 'C': 1, 'p': 0.4, 'k': float('inf')}
        agent1_spec = {'type': 'transformer', 'kwargs': model_kwargs, 'num_simgames':  1, 'max_simmoves': 1, 'C': 1, 'p': 0.4, 'k': float('inf')}
        self_play_args = {
            'num_workers':2, 'num_tournaments': tournaments_left_to_play, 'agents_spec': [agent0_spec, agent1_spec], 
            'num_games':1, 'starting_state':None, 'max_moves':200, 'save':True, 'result_dest':current_self_play_path
        }

        # Let's play
        print(f'Playing {tournaments_left_to_play} tournaments...')
        %run -i "chess_selfplay.py"
        print(f'Self-play complete.')

    # Extract the checkmates from the current_self_play_dir tournament games and save them in the current_training_round_dir.
    current_round_checkmates_path = os.path.join(root_dir, current_training_round_dir, 'checkmates.pkl')
    if not (os.path.exists(current_round_checkmates_path) and os.path.isfile(current_round_checkmates_path)):
        _ = harvest_checkmates(os.path.join(root_dir, current_training_round_dir))

    latest_model_path = os.path.join(root_dir, current_training_round_dir, 'model.pt')
    if not (os.path.exists(latest_model_path) and os.path.isfile(latest_model_path)):

        # No model saved here yet. Create and train a new model based on the previous k rounds of self-play data.
        # We currently have 591 tournaments saved in baseline and 191 in round1. We could use k = 10 and go from there?
        model_kwargs = {'nlayers':6, 'nheads':3, 'embed_dim':18, 'dk':5, 'device':device, 'load_path':None} # Brand new model
        model = TransformerModel(**model_kwargs)
        optimizer = torch.optim.Adam(model.parameters(), lr=0, weight_decay=0)
        loss_fn = TanhLoss()
        dataset = ChessDataset(root_dir=root_dir, look_back=10, device=device)
        train_set, test_set = torch.utils.data.random_split(dataset, [int(len(dataset)*0.8), len(dataset) - int(len(dataset)*0.8)])
        train_loader = DataLoader(train_set, batch_size=1000, shuffle=True, num_workers=0)
        test_loader = DataLoader(test_set, batch_size=1000, shuffle=True, num_workers=0)
        print(f'Training on {len(train_set):,.0f} examples in {len(train_loader):,.0f} batches.')

        # Train on the data
        model_dest = os.path.join(root_dir, current_training_round_dir, 'model.pt')
        model = train(model, loss_fn, optimizer, train_loader, test_loader, warmup_passes=4, max_lr=1e-4, save_dir=model_dest, stopping=5)
        torch.cuda.empty_cache()
        
    # Create next training round directory containing self_play sub-directory, and start loop from the top.
    next_index = int(os.path.split(current_training_round_dir)[-1].split('_')[-1]) + 1
    next_training_round_dir = f'round_{next_index}'
    print(f'Creating next training round directory {next_training_round_dir}')
    next_training_round_path = os.path.join(root_dir, next_training_round_dir)
    os.mkdir(next_training_round_path)
    next_training_round_self_play_path = os.path.join(next_training_round_path, 'self_play')
    os.mkdir(next_training_round_self_play_path)

cuda
Current: round_9, already played: 80, left to play: -30
Training on 238383 examples in 239 batches.


100%|██████████| 60/60 [00:08<00:00,  7.10it/s]


Pass: 1, train loss: 3.44938, test loss: 3.44535, stopping count: 0


100%|██████████| 60/60 [00:08<00:00,  7.08it/s]


Pass: 2, train loss: 3.44867, test loss: 3.44192, stopping count: 0


...

Pass: 50, train loss: 3.43567, test loss: 3.43158, stopping count: 3


100%|██████████| 60/60 [00:07<00:00,  7.79it/s]


Pass: 51, train loss: 3.43535, test loss: 3.43137, stopping count: 4


100%|██████████| 60/60 [00:07<00:00,  8.05it/s]


Pass: 52, train loss: 3.43559, test loss: 3.43136, stopping count: 5
Creating next training round directory round_10
Current: round_10, already played: 0, left to play: 50
Playing 50 tournaments...
